In [1]:
import os
import os.path as op
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import seaborn as sns
sns.set_theme(style="darkgrid", color_codes=True)
sns.set(font_scale=1.35, style="ticks") #set styling preferences
import statsmodels.api as sm
from scipy import stats
import math
from math import pi
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from scipy.spatial.distance import cdist
from scipy.cluster.vq import kmeans2,vq, whiten
import geopandas as gpd
import h5py
import boto.s3
import glob
import boto3
from zipfile import ZipFile
import shutil
import dask.dataframe as dd
from dask.delayed import delayed
import urllib.request
import gzip
import gcsfs
from urllib.parse import urlparse
from google.cloud import storage
import io

In [2]:
# Read the CSV file into a DataFrame
df_exl = pd.read_excel('C:/Users/nrezaei/Documents/WAV_Scenarios.xlsx')
# Get the paths from the DataFrame
paths = df_exl['Outputs'][0:6].tolist() 

In [3]:
cols_to_use = ['IDMerged', 'tripIndex', 'actStartTime', 'actEndTime','duration_travelling', 'cost_BEAM', 'actStartType', 
               'actEndType', 'duration_walking', 'duration_in_privateCar', 'duration_on_bike', 'duration_in_ridehail', 
              'distance_travelling', 'duration_in_transit', 'distance_walking','distance_bike','distance_ridehail', 
              'distance_privateCar', 'distance_transit', 'mode_choice_planned_BEAM','mode_choice_actual_BEAM',
              'vehicleIds', 'distance_mode_choice', 'replanning_status', 'reason', 'fuel_marginal','BlockGroupStart',
              'startX', 'startY', 'bgid_start', 'tractid_start', 'juris_name_start', 'county_name_start', 'mpo_start', 
               'BlockGroupEnd', 'endX', 'endY', 'bgid_end', 'tractid_end', 'juris_name_end', 'county_name_end', 'mpo_end', 
               'emission_marginal', 'duration_door_to_door', 'waitTime_no_replanning', 'waitTime_replanning', 'actPurpose', 
               'mode_choice_actual_6', 'mode_choice_actual_5', 'mode_choice_actual_4', 'trip_mode_AS_trips', 'logsum_trip_Potential_INEXUS',
               'age', 'income', 'hh_cars', 'TAZ_x', 'origin_x', 'destination_x', 'TAZ_y', 'home_taz', 'auto_ownership', 'home_is_urban', 'home_is_rural', 'DRIVEALONEFREE',
               'DRIVEALONEPAY', 'SHARED2FREE', 'SHARED2PAY', 'SHARED3FREE', 'SHARED3PAY', 'WALK', 'BIKE', 'WALK_LOC', 'WALK_LRF', 
               'WALK_EXP', 'WALK_HVY', 'WALK_COM', 'DRIVE_LOC', 'DRIVE_LRF', 'DRIVE_EXP', 'DRIVE_HVY', 'DRIVE_COM', 'TAXI',
               'TNC_SINGLE', 'TNC_SHARED', 'income_quartiles', 'income_deciles' ] # Specify the columns to read

In [4]:
paths

['gs://beam-core-outputs/wheelchair-feb2023/sfbay-10pop-1fleet/inexus/sfbay_baseline_default-1.0_2021__20230319.csv.gz',
 'gs://beam-core-outputs/wheelchair-feb2023/sfbay-10pop-5fleet/inexus/sfbay_baseline_default-1.0_2021__20230211.csv.gz',
 'gs://beam-core-outputs/wheelchair-feb2023/sfbay-10pop-10fleet/inexus/sfbay_baseline_default-1.0_2021__20230208.csv.gz',
 'gs://beam-core-outputs/wheelchair-feb2023/sfbay-10pop-50fleet/inexus/sfbay_baseline_default-1.0_2021__20230211.csv.gz',
 'gs://beam-core-outputs/wheelchair-feb2023/sfbay-10pop-100fleet/inexus/sfbay_baseline_default-1.0_2021__20230211.csv.gz']

In [5]:
%%time
# Read the CSV files from GCS into pandas DataFrames and add the new columns
dfs = []
for i, path in enumerate(paths):
    df = pd.read_csv(path, compression='gzip')
    df['lever_position_price'] = df_exl.loc[i, 'lever_position_price']
    df['lever_position_fltsz'] = df_exl.loc[i, 'lever_position_fltsz']
    dfs.append(df)

<timed exec>:4: DtypeWarning: Columns (77,131,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,158,167,172,174,175,176,177,179,180,237,238,239,240,249) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:4: DtypeWarning: Columns (77,131,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,158,167,172,174,175,176,177,179,180,237,238,239,240,249) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:4: DtypeWarning: Columns (77,131,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,158,167,172,174,175,176,177,179,180,237,238,239,240,249) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:4: DtypeWarning: Columns (77,131,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,158,167,172,174,175,176,177,179,180,237,238,239,240,249) have mixed types. Specify dtype option on import or set low_me

CPU times: total: 15min 13s
Wall time: 17min 4s


In [6]:
dfs[1].head()

,Unnamed: 0.1,IDMerged,tripIndex,actStartTime,actEndTime,duration_travelling,cost_BEAM,actStartType,actEndType,duration_walking,...,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,income_quartiles,income_deciles,lever_position_price,lever_position_fltsz
0,0,1,577,29606.0,29347.0,259.0,0.000000,othmaint,Home,0.0,...,-2997.413793,-2997.413793,-2997.413793,-8.707141,-8.660355,-9.472380,4thD,9thD,1,0.05
1,1,1,578,31843.0,31777.0,66.0,0.000000,school,othmaint,0.0,...,-2997.414938,-2997.414938,-2997.414938,-9.721550,-9.830480,-8.679114,4thD,9thD,1,0.05
2,2,1,581,54458.0,54154.0,304.0,0.000000,Home,school,0.0,...,-2997.414938,-2997.414938,-2997.414938,-8.875765,-8.981690,-10.107585,4thD,9thD,1,0.05
3,3,2,689,46603.0,43966.0,2637.0,5.856778,atwork,work,605.0,...,-1998.309554,-1998.309554,-1998.309554,-8.406941,-8.130668,-9.076208,4thD,9thD,1,0.05
4,4,2,693,57136.0,56365.0,771.0,1.582946,othmaint,atwork,53.0,...,-1998.309554,-1998.309554,-1998.309554,-8.991495,-8.569634,-9.443968,4thD,9thD,1,0.05


In [7]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [18]:
def generate_summary_table_dis(df):
    df['year'] = 2018
    df['socialCarbonCost'] = df['emission_marginal']*185
    df['incomeInThousands'] = df['income']/1000
        
    # Mean pivot table
    mean_inc_table = pd.pivot_table(df,index=['in_wheelchair'], aggfunc={ 
           'incomeInThousands': np.mean,
           'logsum_trip_Potential_INEXUS': np.mean,
            'duration_travelling': np.mean,
            'duration_door_to_door': np.mean,
            'duration_walking': np.mean,
            'duration_in_privateCar': np.mean,
            'duration_on_bike': np.mean,
            'duration_in_ridehail': np.mean,
            'duration_in_transit': np.mean,
            'waitTime_no_replanning': np.mean,
            'waitTime_replanning': np.mean,
            'distance_travelling': np.mean,
            'distance_walking': np.mean,
            'distance_bike': np.mean,
            'distance_ridehail': np.mean,
            'distance_privateCar': np.mean,
            'distance_transit': np.mean,
            'distance_mode_choice': np.mean,
            'replanning_status': np.mean,
            'fuel_marginal': np.mean,
            'emission_marginal': np.mean,
            'cost_BEAM': np.mean,
            'lever_position_price':lambda x: ', '.join(set(x.dropna().astype(str))),
            'lever_position_fltsz':lambda x: ', '.join(set(x.dropna().astype(str))),
            'socialCarbonCost':np.mean,
            'IDMerged': 'nunique',
            'tripIndex': 'nunique'}).reset_index()
    # Median summary table
    median_inc_table = pd.pivot_table(df, index=['in_wheelchair'], aggfunc={
           'incomeInThousands': np.median,
           'logsum_trip_Potential_INEXUS': np.median,
            'duration_travelling': np.median,
            'duration_door_to_door': np.median,
            'duration_walking':np.median,
            'duration_in_privateCar': np.median,
            'duration_on_bike': np.median,
            'duration_in_ridehail': np.median,
            'duration_in_transit': np.median,
            'waitTime_no_replanning': np.median,
            'waitTime_replanning': np.median,
            'distance_travelling': np.median,
            'distance_walking': np.median,
            'distance_bike': np.median,
            'distance_ridehail': np.median,
            'distance_privateCar': np.median,
            'distance_transit': np.median,
            'distance_mode_choice': np.median,
            'replanning_status': np.median,
            'fuel_marginal': np.median,
            'emission_marginal': np.median,
            'cost_BEAM': np.median,
            'socialCarbonCost':np.median,
           }).reset_index() 
   # Sum summary table
    sum_inc_table = pd.pivot_table(df, index=['in_wheelchair'], aggfunc={
            'duration_travelling': np.sum,
            'duration_door_to_door': np.sum,
            'duration_walking': np.sum,
            'duration_in_privateCar': np.sum,
            'duration_on_bike': np.sum,
            'duration_in_ridehail': np.sum,
            'duration_in_transit': np.sum,
            'waitTime_no_replanning': np.sum,
            'waitTime_replanning': np.sum,
            'distance_travelling': np.sum,
            'distance_walking': np.sum,
            'distance_bike': np.sum,
            'distance_ridehail': np.sum,
            'distance_privateCar': np.sum,
            'distance_transit': np.sum,
            'distance_mode_choice': np.sum,
            'replanning_status': np.sum,
            'fuel_marginal': np.sum,
            'emission_marginal': np.sum,
            'cost_BEAM': np.sum,
            'socialCarbonCost':np.sum, 
            }).reset_index() #'in_wheelchair': lambda x: sum(x == True)                                                        
            
    mode_counts = df.groupby(['in_wheelchair', 'mode_choice_actual_BEAM'])['mode_choice_actual_BEAM'].count().unstack().add_prefix('mode_').reset_index()
    sum_inc_table = sum_inc_table.merge(mode_counts, on='in_wheelchair').assign(mode_ridehail_total = lambda x: x['mode_ride_hail'] + x['mode_ride_hail_pooled'])
    sum_inc_table.columns = [col + '_sum' for col in sum_inc_table.columns]
    sum_inc_table = sum_inc_table.rename(columns={'in_wheelchair_sum': 'in_wheelchair'})
    mm_inc_table = pd.merge(mean_inc_table, median_inc_table, on='in_wheelchair', suffixes=('_mean', '_median'))
    summary_inc_table = pd.merge(mm_inc_table, sum_inc_table, on='in_wheelchair', how='left')
    summary_inc_table = summary_inc_table.rename(columns={'tripIndex': 'n_trips', 
                                                      'IDMerged': 'n_agents', 
                                                      'logsum_trip_Potential_INEXUS_median': 'Potential_INEXUS_median',
                                                      'logsum_trip_Potential_INEXUS_mean': 'Potential_INEXUS_mean',
                                                      'mode_ride_hail_sum': 'mode_ride_hail_solo_sum'})        
    
    summary_inc_table = summary_inc_table.iloc[:, :2].join(summary_inc_table.iloc[:, 2:].sort_index(axis=1)) 
    
    # shift column 'person' to first position
    third_column = summary_inc_table.pop('n_trips')
    
    # insert column using insert(position,column_name,first_column) function
    summary_inc_table.insert(2, 'n_trips', third_column)
    
    # calculate median and mean wait times by mode and incRank
    grouped_waitTime_no_replanning = df.groupby(['mode_choice_actual_BEAM', 'in_wheelchair'])['waitTime_no_replanning'].agg(['mean', 'median']).reset_index()
    
    # pivot the wait time statistics by incRank and mode
    waittime_pivot = pd.pivot_table(grouped_waitTime_no_replanning, index=['in_wheelchair'], columns=['mode_choice_actual_BEAM'],
    values=['median', 'mean'], aggfunc=np.sum)
    
    # flatten the multi-index column names and rename them with appropriate suffixes
    waittime_pivot.columns = [f"waitTime_no_replanning_{mode}_{agg}" for (agg, mode) in waittime_pivot.columns.to_flat_index()]
    
    # merge the wait time statistics pivot table with the original pivot table
    summary_inc_table = summary_inc_table.merge(waittime_pivot, on='in_wheelchair', how='left')
    
    # calculate median and mean wait times by mode and incRank
    grouped_waitTime_replanning = df.groupby(['mode_choice_planned_BEAM', 'in_wheelchair'])['waitTime_replanning'].agg(['mean', 'median']).reset_index()
    
    # pivot the wait time statistics by incRank and mode
    waittime_pivot = pd.pivot_table(grouped_waitTime_replanning, index=['in_wheelchair'], columns=['mode_choice_planned_BEAM'], values=['median', 'mean'], aggfunc=np.sum)
    
    # flatten the multi-index column names and rename them with appropriate suffixes
    waittime_pivot.columns = [f"waitTime_replanning_{mode}_{agg}" for (agg, mode) in waittime_pivot.columns.to_flat_index()]
    
    # merge the wait time statistics pivot table with the original pivot table
    summary_inc_table = summary_inc_table.merge(waittime_pivot, on='in_wheelchair', how='left')
    return summary_inc_table

In [19]:
%%time
dfs_summary = []
for df in dfs:
    df_summary = generate_summary_table_dis(df)
    dfs_summary.append(df_summary)

CPU times: total: 31.6 s
Wall time: 31.6 s


In [21]:
%%time
sf_stacked = pd.concat(dfs_summary)

CPU times: total: 0 ns
Wall time: 1.93 ms


In [22]:
sf_stacked

,in_wheelchair,n_agents,n_trips,Potential_INEXUS_mean,Potential_INEXUS_median,cost_BEAM_mean,cost_BEAM_median,cost_BEAM_sum,distance_bike_mean,distance_bike_median,distance_bike_sum,distance_mode_choice_mean,distance_mode_choice_median,distance_mode_choice_sum,distance_privateCar_mean,distance_privateCar_median,distance_privateCar_sum,distance_ridehail_mean,distance_ridehail_median,distance_ridehail_sum,distance_transit_mean,distance_transit_median,distance_transit_sum,distance_travelling_mean,distance_travelling_median,distance_travelling_sum,distance_walking_mean,distance_walking_median,distance_walking_sum,duration_door_to_door_mean,duration_door_to_door_median,duration_door_to_door_sum,duration_in_privateCar_mean,duration_in_privateCar_median,duration_in_privateCar_sum,duration_in_ridehail_mean,duration_in_ridehail_median,duration_in_ridehail_sum,duration_in_transit_mean,duration_in_transit_median,duration_in_transit_sum,duration_on_bike_mean,duration_on_bike_median,duration_on_bike_sum,duration_travelling_mean,duration_travelling_median,duration_travelling_sum,duration_walking_mean,duration_walking_median,duration_walking_sum,emission_marginal_mean,emission_marginal_median,emission_marginal_sum,fuel_marginal_mean,fuel_marginal_median,fuel_marginal_sum,incomeInThousands_mean,incomeInThousands_median,lever_position_fltsz,lever_position_price,mode_bike_sum,mode_bike_transit_sum,mode_car_hov2_sum,mode_car_hov3_sum,mode_car_sum,mode_drive_transit_sum,mode_hov2_teleportation_sum,mode_hov3_teleportation_sum,mode_ride_hail_pooled_sum,mode_ride_hail_solo_sum,mode_ridehail_total_sum,mode_walk_sum,mode_walk_transit_sum,replanning_status_mean,replanning_status_median,replanning_status_sum,socialCarbonCost_mean,socialCarbonCost_median,socialCarbonCost_sum,waitTime_no_replanning_mean,waitTime_no_replanning_median,waitTime_no_replanning_sum,waitTime_replanning_mean,waitTime_replanning_median,waitTime_replanning_sum,waitTime_no_replanning_bike_mean,waitTime_no_replanning_bike_transit_mean,waitTime_no_replanning_car_mean,waitTime_no_replanning_car_hov2_mean,waitTime_no_replanning_car_hov3_mean,waitTime_no_replanning_drive_transit_mean,waitTime_no_replanning_hov2_teleportation_mean,waitTime_no_replanning_hov3_teleportation_mean,waitTime_no_replanning_ride_hail_mean,waitTime_no_replanning_ride_hail_pooled_mean,waitTime_no_replanning_walk_mean,waitTime_no_replanning_walk_transit_mean,waitTime_no_replanning_bike_median,waitTime_no_replanning_bike_transit_median,waitTime_no_replanning_car_median,waitTime_no_replanning_car_hov2_median,waitTime_no_replanning_car_hov3_median,waitTime_no_replanning_drive_transit_median,waitTime_no_replanning_hov2_teleportation_median,waitTime_no_replanning_hov3_teleportation_median,waitTime_no_replanning_ride_hail_median,waitTime_no_replanning_ride_hail_pooled_median,waitTime_no_replanning_walk_median,waitTime_no_replanning_walk_transit_median,waitTime_replanning_bike_mean,waitTime_replanning_bike_transit_mean,waitTime_replanning_car_mean,waitTime_replanning_car_hov2_mean,waitTime_replanning_car_hov3_mean,waitTime_replanning_drive_transit_mean,waitTime_replanning_hov2_teleportation_mean,waitTime_replanning_hov3_teleportation_mean,waitTime_replanning_ride_hail_mean,waitTime_replanning_ride_hail_pooled_mean,waitTime_replanning_walk_mean,waitTime_replanning_walk_transit_mean,waitTime_replanning_bike_median,waitTime_replanning_bike_transit_median,waitTime_replanning_car_median,waitTime_replanning_car_hov2_median,waitTime_replanning_car_hov3_median,waitTime_replanning_drive_transit_median,waitTime_replanning_hov2_teleportation_median,waitTime_replanning_hov3_teleportation_median,waitTime_replanning_ride_hail_median,waitTime_replanning_ride_hail_pooled_median,waitTime_replanning_walk_median,waitTime_replanning_walk_transit_median
0,False,634730,2461285,-0.444347,-0.233815,1.162879,0.502130,2.862176e+06,57.264375,0.0,1.409439e+08,11452.486990,6780.068000,2.818783e+10,10022.812523,5689.343,2.466900e+10,120.808402,0.0,2.97

In [25]:
sf_stacked.to_csv('C:/Users/nrezaei/Documents/sf_2018_stacked_WAV_dis.csv', index=False)

In [24]:
sf_stacked.to_csv('s3://beam-core-act/deepDive/CleanData/SanFrancisco/Stacked/sf_2018_stacked_WAV_incRank_dis.csv', index=False)